# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
def FindingPostsWithNegativeSentiments():
    df = pd.read_csv('../dataset/SO_Workflow_Data.csv')
    my_value = -1
    data = df.loc[df["RatingsGPTFineTuned"] == my_value]

    df = data[data['Body'].str.contains('luigi', case=False)] 
    print(len(df))
    df.to_csv('../dataset/luigi_posts.csv', index=False)

FindingPostsWithNegativeSentiments()

96


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/luigi_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Title","Body"]].apply("-".join, axis=1)
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

96


In [4]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['writing output to files from python luigi-I just tried to run the python '
 'luigi example from the documentation: class TaskA(luigi.Task): def '
 'output(self): return luigi.LocalTarget(xyz) class '
 'FlipLinesBackwards(luigi.Task): def requires(self): return TaskA() def '
 'output(self): return luigi.LocalTarget(abc) def run(self): f = '
 'self.input().open(r) # this will return a file stream that reads from "xyz" '
 'g = self.output().open(w) for line in f: g.write(%s\\n, '
 '.join(reversed(line.strip().split()))) g.close() # needed because files are '
 'atomic I ran it using command line: python Luigi_Test.py FlipLinesBackwards '
 '--local-scheduler I was under the impression that this would create a file '
 'in the directory I am running it in but it doesnt? Am I doing something '
 'wrong? ',
 'Using Parameters in python luigi-I have am triggering Luigi via '
 'luigi.run(["--local-scheduler"], main_task_cls=Test(Server = ActiveServer, '
 'Database = DB)) and in my class I have: 

# Tokenize words and Clean-up text

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['writing', 'output', 'to', 'files', 'from', 'python', 'luigi', 'just', 'tried', 'to', 'run', 'the', 'python', 'luigi', 'example', 'from', 'the', 'documentation', 'class', 'taska', 'luigi', 'task', 'def', 'output', 'self', 'return', 'luigi', 'localtarget', 'xyz', 'class', 'luigi', 'task', 'def', 'requires', 'self', 'return', 'taska', 'def', 'output', 'self', 'return', 'luigi', 'localtarget', 'abc', 'def', 'run', 'self', 'self', 'input', 'open', 'this', 'will', 'return', 'file', 'stream', 'that', 'reads', 'from', 'xyz', 'self', 'output', 'open', 'for', 'line', 'in', 'write', 'join', 'reversed', 'line', 'strip', 'split', 'close', 'needed', 'because', 'files', 'are', 'atomic', 'ran', 'it', 'using', 'command', 'line', 'python', 'luigi_test', 'py', 'local', 'scheduler', 'was', 'under', 'the', 'impression', 'that', 'this', 'would', 'create', 'file', 'in', 'the', 'directory', 'am', 'running', 'it', 'in', 'but', 'it', 'doesnt', 'am', 'doing', 'something', 'wrong']]


In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['write', 'output', 'file', 'try', 'run', 'documentation', 'class', 'taska', 'def', 'output', 'self', 'return', 'localtarget', 'def', 'require', 'self', 'return', 'def', 'output', 'self', 'return', 'run', 'self', 'self', 'input', 'open', 'return', 'file', 'stream', 'read', 'xyz', 'self', 'output', 'open', 'line', 'write', 'join', 'reverse', 'line', 'strip', 'split', 'close', 'need', 'file', 'run', 'use', 'command', 'line', 'luigi_t', 'local', 'scheduler', 'impression', 'create', 'file', 'directory', 'running', 'wrong']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 1), (7, 4), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 4), (18, 1), (19, 1), (20, 4), (21, 1), (22, 3), (23, 1), (24, 1), (25, 6), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1)]]


In [10]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('class', 1),
  ('close', 1),
  ('command', 1),
  ('create', 1),
  ('def', 3),
  ('directory', 1),
  ('documentation', 1),
  ('file', 4),
  ('impression', 1),
  ('input', 1),
  ('join', 1),
  ('line', 3),
  ('local', 1),
  ('localtarget', 1),
  ('luigi_t', 1),
  ('need', 1),
  ('open', 2),
  ('output', 4),
  ('read', 1),
  ('require', 1),
  ('return', 4),
  ('reverse', 1),
  ('run', 3),
  ('running', 1),
  ('scheduler', 1),
  ('self', 6),
  ('split', 1),
  ('stream', 1),
  ('strip', 1),
  ('taska', 1),
  ('try', 1),
  ('use', 1),
  ('write', 2),
  ('wrong', 1),
  ('xyz', 1)]]

In [10]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [11]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]

100%|██████████| 540/540 [09:45<00:00,  1.08s/it]


In [12]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
297,100% Corpus,2,asymmetric,0.61,0.544267
299,100% Corpus,2,asymmetric,symmetric,0.544267
298,100% Corpus,2,asymmetric,0.9099999999999999,0.542629
411,100% Corpus,6,symmetric,0.31,0.537383
391,100% Corpus,6,0.01,0.31,0.537121
...,...,...,...,...,...
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.434032
423,100% Corpus,7,0.01,0.9099999999999999,0.431641
453,100% Corpus,8,0.01,0.9099999999999999,0.425680
473,100% Corpus,8,symmetric,0.9099999999999999,0.422407


In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -6.296507282784464

Coherence Score:  0.3406418101810571


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.092721  0.051365       1        1  40.163327
3      0.066867 -0.061547       2        1  33.759423
1     -0.024559  0.007730       3        1   9.309453
4     -0.001401  0.003361       4        1   8.998513
5     -0.051273 -0.016972       5        1   5.280212
0     -0.082354  0.016063       6        1   2.489071, topic_info=          Term        Freq       Total Category  logprob  loglift
25        self  454.000000  454.000000  Default  30.0000  30.0000
119       task  399.000000  399.000000  Default  29.0000  29.0000
24   scheduler   80.000000   80.000000  Default  28.0000  28.0000
416       user   73.000000   73.000000  Default  27.0000  27.0000
102        pid   46.000000   46.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
30         try    0.846868   62.428134   Topic6  -5.9664  -0.6070
57       build    0.631296   12.814042   Topic6  -6.2602   0.6827
7         file    0.858658  278.298645   Topic6  -5.9526  -2.0878
171        txt    0.581791   14.110755   Topic6  -6.3419   0.5047
105      print    0.593891   27.450858   Topic6  -6.3213  -0.1402

[365 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1196      1  0.813490     action
1196      2  0.090388     action
1575      1  0.224769  afternoon
1575      2  0.224769  afternoon
1575      6  0.449539  afternoon
...     ...       ...        ...
1160      1  0.258498        yet
1160      2  0.258498        yet
721       1  0.033720      yield
721       2  0.910431      yield
721       3  0.033720      yield

[858 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 5, 6, 1])

In [12]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.006*"morning" + 0.005*"branch" + 0.005*"afternoon" + 0.005*"luigid" + '
  '0.005*"scheduler" + 0.004*"run" + 0.004*"luigi" + 0.004*"connect" + '
  '0.004*"docker" + 0.004*"task"'),
 (1,
  '0.013*"luigi" + 0.013*"file" + 0.013*"self" + 0.008*"use" + 0.008*"try" + '
  '0.008*"error" + 0.007*"test" + 0.007*"output" + 0.007*"get" + '
  '0.006*"return"'),
 (2,
  '0.036*"file" + 0.030*"self" + 0.026*"run" + 0.025*"task" + 0.019*"worker" + '
  '0.016*"line" + 0.013*"user" + 0.012*"luigi" + 0.012*"return" + '
  '0.011*"site_package"'),
 (3,
  '0.054*"self" + 0.050*"task" + 0.029*"run" + 0.023*"def" + 0.021*"output" + '
  '0.017*"return" + 0.015*"class" + 0.014*"require" + 0.014*"luigi" + '
  '0.013*"file"'),
 (4,
  '0.021*"task" + 0.019*"self" + 0.017*"run" + 0.011*"return" + 0.010*"pid" + '
  '0.010*"scheduler" + 0.009*"worker" + 0.009*"def" + 0.007*"time" + '
  '0.007*"debug"'),
 (5,
  '0.022*"self" + 0.011*"task" + 0.008*"run" + 0.007*"luigi" + '
  '0.006*"startdate" + 0.006*"para

In [13]:
num_topics = 6
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicLuigi.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0  31467892          writing output to files from python luigi   
1  31533967                   Using Parameters in python luigi   
2  32978429  python luigi died unexpectedly with exit code -11   
3  33332058                     Luigi Pipeline beginning in S3   
4  34147832  output for append job in BigQuery using Luigi ...   

                                                Body  RatingsSentiCR  \
0  I just tried to run the python luigi example f...              -1   
1  I have am triggering Luigi via\n\nluigi.run(["...              -1   
2  I have a data pipeline with luigi that works p...               1   
3  My initial files are in AWS S3. Could someone ...              -1   
4  I have a Bigquery task which only aims to appe...               0   

   RatingsGPT35  RatingsGPTFineTuned  \
0            -1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   
3         